In [44]:
from pandas import DataFrame
from DATA225utils import make_connection, dataframe_query

In [45]:
def make_table(table, sql):
    cursor.execute(f"DROP TABLE IF EXISTS {table}")
    cursor.execute(sql)

In [46]:
def display_table(table, order_by=''):
    sql = f"SELECT * FROM {table}"
    
    if order_by != '':
        sql = sql + " ORDER BY " + order_by
        
    _, df = dataframe_query(cursor, sql)    
    return df

In [47]:
conn = make_connection(config_file = 'Analytical_db.ini')
cursor = conn.cursor()

In [41]:
conn_warehouse = make_connection(config_file = 'techorcas_wh.ini')
cursor_warehouse = conn_warehouse.cursor()

Exception: Configuration file 'techorcas_wh.ini' doesn't exist.

## Date Dimension Table

In [69]:
#Dropping Churn_Customers table if already exists.
cursor.execute("DROP TABLE IF EXISTS Date_Dimension")

In [70]:
sql = ( """
        CREATE TABLE Date_Dimension
        (   
           
            Date DATE ,
            Day varchar(10),
            Month varchar(10),
            Year int,
            Quarter varchar(10),
            PRIMARY KEY (DATE)
        )
        """
      )

cursor.execute(sql)

In [71]:
#Insert vales into Churn_Customers
sql = ( """
        INSERT INTO Date_Dimension
        VALUES (%s, %s, %s, %s, %s )
        """
      )

In [72]:
def transform(row):
    if row[1] == 'NA':
        row[1] = 0


In [73]:
import csv
first = True
i = 0

with open(r'Date_Dimension_Table.csv', newline='') as csv_file:
    data = csv.reader(csv_file, delimiter=',', quotechar='"')
    
    for row in data:
        if not first:
            transform(row)
            cursor.execute(sql, row)

            
        first = False
    
conn.commit()

## Product Dimension Table

In [74]:
#Dropping Churn_Customers table if already exists.
cursor.execute("DROP TABLE IF EXISTS Product_Dimension")

In [75]:
sql = ( """
        CREATE TABLE Product_Dimension
        (   
            SKU VARCHAR(150),
            Style  VARCHAR(50),
            color VARCHAR(50),
            category VARCHAR(50),
            size VARCHAR(10),
            PRIMARY KEY (SKU)
        )
        """
      )

make_table('Product_Dimension', sql)

In [76]:
#Insert vales into Churn_Customers
sql = ( """
        INSERT INTO Product_Dimension
        VALUES (%s, %s, %s, %s, %s )
        """
      )

In [77]:
import csv
first = True
i = 0

with open(r'Product_Dimensional_Table.csv', newline='') as csv_file:
    data = csv.reader(csv_file, delimiter=',', quotechar='"')
    
    for row in data:
        if not first:
            transform(row)
            cursor.execute(sql, row)

            
        first = False
    
conn.commit()

# 	Location Dimension Table

In [78]:
#Dropping Churn_Customers table if already exists.
cursor.execute("DROP TABLE IF EXISTS Location_Dimension_Table")

In [79]:
sql = ( """
        CREATE TABLE Location_Dimension_Table
        (   
            ship_postal_code int,
            ship_country VARCHAR(50),
            ship_state VARCHAR(50),
            ship_city VARCHAR(50),
            PRIMARY KEY (ship_postal_code)
        )
        """
      )

make_table('Location_Dimension_Table', sql)


In [80]:
#Insert vales into Churn_Customers
sql = ( """
        INSERT INTO Location_Dimension_Table
        VALUES (%s, %s, %s, %s )
        """
      )

In [81]:
import csv
first = True
i = 0

with open(r'Location_Dimension_Table.csv', newline='') as csv_file:
    data = csv.reader(csv_file, delimiter=',', quotechar='"')
    
    for row in data:
        if not first:
            transform(row)
            cursor.execute(sql, row)

            
        first = False
    
conn.commit()

## 	Fulfilment Dimension Table

In [82]:
#Dropping Churn_Customers table if already exists.
cursor.execute("DROP TABLE IF EXISTS Fulfilment_Dimension_Table")

In [83]:
sql = ( """
        CREATE TABLE Fulfilment_Dimension_Table
        (   
Order_ID VARCHAR(150),
ship_service_level VARCHAR(150),
fulfilled_by VARCHAR(150),
Sales_Channel VARCHAR(150),
Fulfilment VARCHAR(150),
promotion_ids VARCHAR(3000),
Courier_Status VARCHAR(150),
status VARCHAR(150),

            PRIMARY KEY (Order_ID)
        )
        """
      )

make_table('Fulfilment_Dimension_Table', sql)


In [84]:
#Insert vales into Churn_Customers
sql = ( """
        INSERT INTO Fulfilment_Dimension_Table
        VALUES (%s, %s, %s, %s , %s, %s, %s, %s)
        """
      )

In [85]:
import csv
first = True
i = 0

with open(r'Fulfilment_Dimension_Table - Copy.csv', newline='') as csv_file:
    data = csv.reader(csv_file, delimiter=',', quotechar='"')
    
    for row in data:
        if not first:
            transform(row)
            cursor.execute(sql, row)

            
        first = False
    
conn.commit()

## Sales Fact Table

In [94]:
#Dropping Churn_Customers table if already exists.
cursor.execute("DROP TABLE IF EXISTS Sales_Fact_Table")

In [95]:
sql = ( """
        CREATE TABLE Sales_Fact_Table
        (   
Order_ID VARCHAR(150),
SKU VARCHAR(150),
Date DATE ,
Amount int,
B2B VARCHAR(50),
Inventory int NULL,
PRIMARY KEY (Order_ID,sku)
        )
        """
      )

make_table('Sales_Fact_Table', sql)


In [96]:
#Insert vales into Churn_Customers
sql = ( """
        INSERT INTO Sales_Fact_Table
        VALUES (%s, %s, %s, %s , %s, %s)
        """
      )

In [97]:
import csv
first = True
i = 0

with open(r'Sales_Fact_Table.csv', newline='') as csv_file:
    data = csv.reader(csv_file, delimiter=',', quotechar='"')
    
    for row in data:
        if not first:
            transform(row)
            cursor.execute(sql, row)

            
        first = False
    
conn.commit()

## Shipping_Fact_Table

In [98]:

#Dropping Churn_Customers table if already exists.
cursor.execute("DROP TABLE IF EXISTS Shipping_Fact_Table")

In [99]:
sql = ( """
        CREATE TABLE Shipping_Fact_Table
        (   
Order_ID VARCHAR(150),	
ship_postal_code int,	
Quantity int,

PRIMARY KEY (Order_ID)
        )
        """
      )

make_table('Shipping_Fact_Table', sql)


In [100]:
#Insert vales into Churn_Customers
sql = ( """
        INSERT INTO Shipping_Fact_Table
        VALUES (%s, %s, %s)
        """
      )

In [101]:
import csv
first = True
i = 0

with open(r'Shipping_Fact_Table.csv', newline='') as csv_file:
    data = csv.reader(csv_file, delimiter=',', quotechar='"')
    
    for row in data:
        if not first:
            transform(row)
            cursor.execute(sql, row)

            
        first = False
    
conn.commit()

## Queries

In [102]:
import mysql.connector
import pandas as pd

#### Query 1

In [103]:
myresult = pd.read_sql('SELECT s.SKU, P.style, P.color, s.amount FROM Sales_Fact_Table s JOIN Product_Dimension P ON s.SKU = P.SKU',conn)

table=pd.DataFrame(myresult)
table=table.reset_index(drop=True)
pd.set_option('expand_frame_repr', False)
print(table)


C:\Users\VENKATACHALAM N\AppData\Local\Temp\ipykernel_17728\76162046.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  myresult = pd.read_sql('SELECT s.SKU, P.style, P.color, s.amount FROM Sales_Fact_Table s JOIN Product_Dimension P ON s.SKU = P.SKU',conn)


                     SKU     style   color  amount
0      JNE2032-KR-205-XL   JNE2032   Black     301
1             J0097-KR-M     J0097   Green     544
2        JNE3440-KR-N-XS   JNE3440   Brown     422
3            J0373-KR-XS     J0373  Purple     558
4         SET324-KR-NP-S    SET324   Black     635
...                  ...       ...     ...     ...
37691        J0014-LCD-M     J0014  Purple       0
37692        J0009-SKD-M     J0009   Black       0
37693  PJNE2171-KR-N-5XL  PJNE2171     Red       0
37694        J0012-SKD-L     J0012     Red       0
37695      J0012-SKD-XXL     J0012   White       0

[37696 rows x 4 columns]


#### Query 2

In [ ]:
myresult = pd.read_sql("SELECT d.day, d.month, d.year FROM Sales_Fact_Table s JOIN Date_Dimension d ON s.Date = d.Date where order_Id= '171-9918163-5396330';",conn)

table=pd.DataFrame(myresult)
table=table.reset_index(drop=True)
pd.set_option('expand_frame_repr', False)
print(table)

#### Query 3

In [ ]:
myresult = pd.read_sql('Select s.order_Id, f.fulfilment, f.Courier_Status, s.Quantity from shipping_fact_table s join Fulfilment_Dimension_Table f on s.order_id=f.order_id',conn)

table=pd.DataFrame(myresult)
table=table.reset_index(drop=True)
pd.set_option('expand_frame_repr', False)
print(table)

#### Query 4

In [ ]:
myresult = pd.read_sql('Select s.order_id , l.ship_postal_code, l.ship_country, l.ship_state, l.ship_city from shipping_fact_table s join location_dimension_table l on s.Ship_postal_code = l.ship_postal_code',conn)

table=pd.DataFrame(myresult)
table=table.reset_index(drop=True)
pd.set_option('expand_frame_repr', False)
print(table)

In [104]:
cursor.close()
conn.close()